#Pretraining Data Generation Script

This notebook processes tsv data and uploads the processed data to GCS to be used for pretraining Mutformer.

Note: Run multiple copies of this notebook in multiple VMs to generate data in parallel for multiple models

* Note: TO ACCESS ANY BUCKET WITH PERMISSION TO VIEW: go to this address: https://console.cloud.google.com/storage/browser/(BUCKET_NAME)



#Downgrade Python and Tensorflow 

(the default python version in Colab does not support Tensorflow 1.15)

* **Note** that because the Python used in this notebook is not the default path, syntax highlighting most likely will not function.

####1. First, download and install Python version 3.7:

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_22.11.1-1-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "python3" --user

--2023-04-04 16:46:30--  https://repo.anaconda.com/miniconda/Miniconda3-py37_22.11.1-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86308321 (82M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  82.31M   100MB/s    in 0.8s    

2023-04-04 16:46:31 (100 MB/s) - ‘mini.sh’ saved [86308321/86308321]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...





Preparing transaction: - \ | / - \ done
Executing transaction: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when runn

####2. Then, reload the webpage (not restart runtime) to allow Colab to recognize the newly installed python
####3. Finally, run the following command to install tensorflow 1.15:

In [ ]:
!python3 -m pip install tensorflow==1.15
!python3 -m pip install protobuf==3.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 7.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1


# Configure settings

In [ ]:
#@markdown ###General Config
#@markdown Whether or not this script is being run in a GCP runtime (if more memory is required for large databases):
GCP_RUNTIME = False #@param {type:"boolean"}
#@markdown Name of the GCS bucket to use (Make sure to set this to the name of your own GCS  bucket):
BUCKET_NAME = "" #@param {type:"string"}
BUCKET_PATH = "gs://"+BUCKET_NAME
#@markdown Because of dynamic masking, which requires generating a new dataset for each train epoch, the train, eval, and test set are written into separate folders.
#@markdown * Folder in GCS to store pretraining data in (dynamic masking generation write multiple datasets as subfolders inside of this folder):
DATA_DIR = "pretraining_data_1024_embedded_mutformer" #@param {type:"string"}
#@markdown \
#@markdown 
#@markdown 
#@markdown ###Data Config
#@markdown Maximum number of datasets to keep at a time (make sure this value is the same in the training script):
DATA_COPIES = 20 #@param {type:"integer"}
#@markdown Maximum output data sequence length:
MAX_SEQ_LENGTH =  1024#@param {type:"integer"}
#@markdown For the masked LM task, a certain number of amino acids per sequence are masked. This number will be determined below by either a probability or a fixed number of masks, whichever number is lower.
#@markdown * What probability to use for masking amino acids:
MASKED_LM_PROB = 0.15 #@param
#@markdown * What fixed max number of masked amino acids to use:
MAX_PREDICTIONS = 20 #@param {type:"integer"}

DATA_INFO = {      ##dictionary that will be uploaded alongside each dataset to indicate its parameters
      "sequence_length":MAX_SEQ_LENGTH,
      "max_num_predictions":MAX_PREDICTIONS,
      "max_masked_prob":MASKED_LM_PROB
}
#### Vocabulary for the model (MutFormer uses the vocabulary below) ([PAD]
#### [UNK],[CLS],[SEP], and [MASK] are necessary default tokens; B and J
#### are markers for the beginning and ending of a protein sequence,
#### respectively; the rest are all amino acids possible, ranked 
#### approximately by frequency of occurence in human population)
#### vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
vocab = \
'''[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
L
S
B
J
E
A
P
T
G
V
K
R
D
Q
I
N
F
H
Y
C
M
W'''
with open("vocab.txt", "w") as fo:
  for token in vocab.split("\n"):
    fo.write(token+"\n")

#If using a GCP runtime to generate data (if database is large and more memory is needed), follow these instructions to set it up

###1) Create a VM from the GCP website
###2) Open a command prompt on your computer and perform the following steps"
To ssh into the VM, run:

```
gcloud beta compute ssh --zone <COMPUTE ZONE> <VM NAME> --project <PROJECT NAME> -- -L 8888:localhost:8888
```

Note: Make sure the port above matches the port below (in this case it's 8888)
\
\
In the new command prompt that popped out, either run each of the commands below individually, or copy and paste the one liner below:
```
sudo apt-get update
sudo apt-get -y install python3 python3-pip
sudo apt-get install pkg-config
sudo apt-get install libhdf5-serial-dev
sudo apt-get install libffi6 libffi-dev
sudo -H pip3 install jupyter tensorflow==1.14 google-api-python-client tqdm
sudo -H pip3 install jupyter_http_over_ws
jupyter serverextension enable --py jupyter_http_over_ws
jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0   --no-browser
```
One command:
```
sudo apt-get update ; sudo apt-get -y install python3 python3-pip ; sudo apt-get install pkg-config ; sudo apt-get -y install libhdf5-serial-dev ; sudo apt-get install libffi6 libffi-dev; sudo -H pip3 install jupyter tensorflow==1.14 google-api-python-client tqdm ; sudo -H pip3 install jupyter_http_over_ws ; jupyter serverextension enable --py jupyter_http_over_ws ; jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0   --no-browser
```
###3) In this notebook, click the "connect to local runtime" option under the connect button, and copy and paste the link outputted by command prompt with "locahost: ..."

#Clone the MutFormer repo

In [ ]:
if GCP_RUNTIME:
  !sudo apt-get -y install git-all
#@markdown Where to clone the repo into:
REPO_DESTINATION_PATH = "mutformer" #@param {type:"string"}
import os,shutil
if not os.path.exists(REPO_DESTINATION_PATH):
  os.makedirs(REPO_DESTINATION_PATH)
else:
  shutil.rmtree(REPO_DESTINATION_PATH)
  os.makedirs(REPO_DESTINATION_PATH)
cmd = "git clone https://github.com/WGLab/mutformer.git \"" + REPO_DESTINATION_PATH + "\""
!{cmd}

Cloning into 'mutformer'...
remote: Enumerating objects: 1574, done.
remote: Counting objects: 100% (454/454), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 1574 (delta 313), reused 364 (delta 256), pack-reused 1120
Receiving objects: 100% (1574/1574), 5.93 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (1102/1102), done.


#Imports/Authenticate for GCP

In [ ]:
if not GCP_RUNTIME:
  def authenticate_user(): ##authentication function that uses link authentication instead of popup
    if os.path.exists("/content/.config/application_default_credentials.json"): 
      return
    print("Authorize for runtime GCS:")
    !gcloud auth login --no-launch-browser
    print("Authorize for TPU GCS:")
    !gcloud auth application-default login  --no-launch-browser
  authenticate_user()

import sys
import json
import random
import logging
import tensorflow.compat.v1 as tf
import re
import time
import os
import shutil
import random
import importlib

if REPO_DESTINATION_PATH == "mutformer":
  if os.path.exists("mutformer_code"):
    shutil.rmtree("mutformer_code")
  shutil.copytree(REPO_DESTINATION_PATH,"mutformer_code")
  REPO_DESTINATION_PATH = "mutformer_code"
if not os.path.exists("mutformer"):
  shutil.copytree(REPO_DESTINATION_PATH+"/mutformer_model_code","mutformer")
else:
  shutil.rmtree("mutformer")
  shutil.copytree(REPO_DESTINATION_PATH+"/mutformer_model_code","mutformer")
if "mutformer" in sys.path:
  sys.path.remove("mutformer")
sys.path.append("mutformer")

from mutformer import tokenization

##reload modules so that you don't need to restart the runtime to reload modules in case that's needed
modules2reload = [tokenization]
for module in modules2reload:
    importlib.reload(module)

#Specify Input Data location/Mount Drive if needed

In [ ]:
if not GCP_RUNTIME:
  from google.colab import drive
#@markdown Input finetuning data folder: data will be read from here to be processed and uploaded to GCS (can be a drive path or a GCS path; must be a GCS path if using GCP_RUNTIME):
INPUT_DATA_FOLDER = "" #@param {type: "string"}
if "/content/drive" in INPUT_DATA_FOLDER:
  !fusermount -u /content/drive
  drive.flush_and_unmount()
  drive.mount('/content/drive', force_remount=True)
  DRIVE_PATH = "/content/drive/My Drive"

#Data Generation

As part of the data generation process, this script:
1. Create shards for eval, test, and train sets: in order to conserve memory during data generation, data is split into multiple shards to be processed and uploaded.
2. Generates eval, test sets:
  * Eval and test sets do not need to be dynamically masked, so they are generated once into seperate folders inside GCS 
3. Generates train set repeatedly for dynamic masking:
  * The script generates DATA_COPIES datasets using the same input train data, but with each epoch containing data with different amino acid masking/alterations, which prevents overfitting during pretraining

## Data preparation (make shards)

In [ ]:
#@markdown Whether or not to store shards into GCS (useful for large databases):
GCS_shards = False #@param {type:"boolean"}
#@markdown If using a data from google drive, size of a single chunk/shard of data (in terms of lines/datatpoints):
chunk_size_gd = 256000 #@param {type:"number"}
#@markdown If using a data from GCS, size of a single chunk/shard of data (in terms of bytes)(the code segment will take care of abnormal line cutoffs):
chunk_size_gcs =  500e6 #@param {type:"number"}
chunk_size_gcs = int(chunk_size_gcs)

def make_shards(dataset):    
  print("Generating shards for "+dataset+":\n")
  if os .path.exists("./shards_tmp_"+dataset): ##data will be written as shards to prevent one single files from getting too large
    shutil.rmtree("./shards_tmp_"+dataset)
  os.makedirs("./shards_tmp_"+dataset)

  if os .path.exists("./shards_"+dataset): 
    shutil.rmtree("./shards_"+dataset)
  os.makedirs("./shards_"+dataset)
  
  if GCS_shards:
    print("removing existing data if it exists...")
    cmd = "gsutil -m rm -r "+INPUT_DATA_FOLDER+"/shards_"+dataset+""
    !{cmd}
  start = 0
  end = chunk_size_gcs
  previous_truncated = ""
  i=0
  while True:
    print("Processing shard "+str(i))
    ##download the selected portion of the input file
    if "gs://" in INPUT_DATA_FOLDER:
      cmd = "gsutil cat -r "+str(start)+"-"+str(end)+" "+INPUT_DATA_FOLDER+"/"+dataset+".txt"+" | gsutil -q cp - ./shards_tmp_"+dataset+"/shard_tmp"
      !{cmd}
    else:
      cmd = "cat -r "+str(start)+"-"+str(end)+" "+INPUT_DATA_FOLDER+"/"+dataset+".txt"+" | gsutil -q cp - ./shards_tmp_"+dataset+"/shard_tmp"
      !{cmd}
    ##get the line count
    cmd = "wc -l <./shards_tmp_"+dataset+"/shard_tmp"
    line_count = !{cmd}
    line_count = int(line_count[0])
    
    ##get the actual byte count
    cmd = "wc -c <./shards_tmp_"+dataset+"/shard_tmp"
    byte_count = !{cmd}
    byte_count = int(byte_count[0])
    if line_count == 0:
      print("(Ignore the previous ServiceException if one was raised.) finished after processing "+str(i+1)+" shards... appending the last truncated line to the end and continuing\n\n")
      i-=1
      break
    print("processing",line_count,"lines...")
    ##get the last few lines of the downloaded file
    cmd = "dd  if=./shards_tmp_"+dataset+"/shard_tmp ibs=1 skip="+str(byte_count-MAX_SEQ_LENGTH*2)+" count="+str(MAX_SEQ_LENGTH*2)+" status=none > previous_tcd.txt"
    !{cmd}
    ##truncate off the last line
    cmd = "sed -ni \'"+str(1)+","+str(line_count)+"p;"+str(line_count)+"q\' ./shards_tmp_"+dataset+"/shard_tmp"
    !{cmd}

    ##add the previously truncated line to the front of the file
    cmd = "sed -i \'1s/^/"+previous_truncated+" /\' ./shards_tmp_"+dataset+"/shard_tmp >garbage.txt"
    !{cmd}
    ##get the last line, which just got truncated, but will be added to the front of the next shard
    previous_truncated = open("previous_tcd.txt").read().split("\n")[-1]
    ##copy data to GCS
    
    if GCS_shards:
      print("Uploading to GCS...\n")
      cmd = "gsutil -q cp ./shards_tmp_"+dataset+"/shard_tmp "+INPUT_DATA_FOLDER+"/shards_"+dataset+"/shard_"+str(i)
      !{cmd}
    else:
      cmd = "cp ./shards_tmp_"+dataset+"/shard_tmp ./shards_"+dataset+"/shard_"+str(i)
      !{cmd}

    start+=chunk_size_gcs
    end+=chunk_size_gcs
    i+=1
  ##appending the last truncated line to the end of the last file

  if GCS_shards:
    cmd = "gsutil -q cp "+INPUT_DATA_FOLDER+"/shards_"+dataset+"/shard_"+str(i)+" ./shards_tmp_"+dataset+"/shard_tmp"
    !{cmd}
    with open("./shards_tmp_"+dataset+"/shard_tmp","a") as writer:
      writer.write(previous_truncated)
    cmd = "gsutil cp ./shards_tmp_"+dataset+"/shard_tmp "+INPUT_DATA_FOLDER+"/shards_"+dataset+"/shard_"+str(i)
    !{cmd}
  else:
    cmd = "cp ./shards_"+dataset+"/shard_"+str(i)+" ./shards_tmp_"+dataset+"/shard_tmp"
    !{cmd}
    with open("./shards_tmp_"+dataset+"/shard_tmp","a") as writer:
      writer.write(previous_truncated)
    cmd = "cp ./shards_tmp_"+dataset+"/shard_tmp ./shards_"+dataset+"/shard_"+str(i)
    !{cmd}
  if GCS_shards:
    data_dir = INPUT_DATA_FOLDER+"/shards_"+dataset
  else:
    data_dir = "./shards_"+dataset
  return data_dir
input_train_dir = make_shards("train")
input_eval_dir = make_shards("eval")
input_test_dir = make_shards("test")

Generating shards for train:

Processing shard 0
processing 150532 lines...
Processing shard 1
(Ignore the previous ServiceException if one was raised.) finished after processing 2 shards... appending the last truncated line to the end and continuing


Generating shards for eval:

Processing shard 0
processing 999 lines...
Processing shard 1
(Ignore the previous ServiceException if one was raised.) finished after processing 2 shards... appending the last truncated line to the end and continuing


Generating shards for test:

Processing shard 0
processing 4553 lines...
Processing shard 1
(Ignore the previous ServiceException if one was raised.) finished after processing 2 shards... appending the last truncated line to the end and continuing




##Define Data Generation Op

In [ ]:
def generate_data(input_dir,dir):
  seed = random.randrange(sys.maxsize)
  input_files = ",".join([input_dir+"/"+file for file in tf.io.gfile.listdir(input_dir)])
  out_files = ",".join([dir+"/"+file+".tfrecord" for file in tf.io.gfile.listdir(input_dir)])
  print("input_files:",input_files,"output_files:",out_files)

  XARGS_CMD = (f"python mutformer/create_pretraining_data.py "
              f"--input_file={input_files} "
              f"--output_file={out_files} "
              f"--vocab_file=vocab.txt "
              f"--do_lower_case=False "
              f"--max_predictions_per_seq={MAX_PREDICTIONS} "
              f"--max_seq_length={MAX_SEQ_LENGTH} "
              f"--masked_lm_prob={MASKED_LM_PROB} "
              f"--random_seed={seed} "
              f"--dupe_factor=1")
  
  if os.path.exists(dir):
    shutil.rmtree(dir)
  os.makedirs(dir)
  
  !$XARGS_CMD

##Eval and Test Data Generation

In [ ]:
EVAL_DIR = f"{DATA_DIR}/eval"
TESTING_DIR = f"{DATA_DIR}/test"

##Eval set
if os.path.exists(EVAL_DIR):
  shutil.rmtree(EVAL_DIR)
os.makedirs(EVAL_DIR)
generate_data(input_eval_dir,EVAL_DIR)
with open(EVAL_DIR+"/info.json","w+") as out: ##writes out the dictionary containing
      json.dump(DATA_INFO,out,indent=2)                   ##the dataset's parameters

##Test set
if os.path.exists(TESTING_DIR):
  shutil.rmtree(TESTING_DIR)
os.makedirs(TESTING_DIR)
generate_data(input_test_dir,TESTING_DIR)
with open(TESTING_DIR+"/info.json","w+") as out: 
      json.dump(DATA_INFO,out,indent=2)                  

cmd="gsutil -m cp -r "+DATA_DIR +" gs://"+BUCKET_NAME
!{cmd}

input_files: ./shards_eval/shard_0 output_files: pretraining_data_1024_embedded_mutformer/eval/shard_0.tfrecord
started
INFO:tensorflow:*** Reading from input files train ***
I0404 17:08:28.146374 139907418386816 create_pretraining_data.py:391] *** Reading from input files train ***
INFO:tensorflow:  ./shards_eval/shard_0
I0404 17:08:28.146642 139907418386816 create_pretraining_data.py:393]   ./shards_eval/shard_0
tokenizing inputs:: 100% 1000/1000 [00:03<00:00, 279.63it/s]
INFO:tensorflow:files processed:1000
I0404 17:08:31.728327 139907418386816 create_pretraining_data.py:209] files processed:1000
creating training instances from document: 100% 1/1 [00:02<00:00,  2.52s/it]
INFO:tensorflow:length of train_instances:1000
I0404 17:08:34.250910 139907418386816 create_pretraining_data.py:400] length of train_instances:1000
INFO:tensorflow:*** Writing to output files ***
I0404 17:08:34.251165 139907418386816 create_pretraining_data.py:403] *** Writing to output files ***
INFO:tensorflow:  

##Repeated Parallel Training Data Generation (for dynamic masking)

In [ ]:
#@markdown When finished generating all DATA_COPIES datasets, how long to wait before checking again if the data has been used (to minimize interaction with GCS, should be around the same time it takes for the mode train script to train 1 model):
CHECK_DATA_EVERY_N_SECS = 1200 #@param {type:"integer"}

TRAIN_DIR = f"{DATA_DIR}/train"

while True:
  try:
    available_indexes = tf.io.gfile.listdir(BUCKET_PATH+"/"+TRAIN_DIR) ##get all currently 
  except Exception:                                                    ##generated datasets
    available_indexes = []

  print("Already generated datasets:",available_indexes)
   
  for i in range(0,DATA_COPIES):                                     
    if os.path.exists(TRAIN_DIR):
      shutil.rmtree(TRAIN_DIR)
    
    os.makedirs(TRAIN_DIR)
    available_indexes = [''.join([i for i in available_index if i.isdigit()]) for available_index in available_indexes]
    if str(i) not in available_indexes:
      print("Processing data for dataset number:",i)
      out_dir = TRAIN_DIR+"/"+str(i)
      print("Writing into dir:",out_dir)
      if not os.path.exists(out_dir):
        os.makedirs(out_dir)
      generate_data(input_train_dir,out_dir)
      print("\nUpdating and uploading data info json...\n")
      
      with open(out_dir+"/info.json","w+") as out: ##writes out a dictionary containing
        json.dump(DATA_INFO,out,indent=2)                   ##the dataset's parameters
      print("Data info json uploaded successfully")
      cmd="gsutil -m cp -r "+out_dir+" "+BUCKET_PATH+"/"+out_dir
      !{cmd}
      available_indexes = tf.io.gfile.listdir(BUCKET_PATH+"/"+TRAIN_DIR) ##we only have to refresh 
                                                                        ##available indexes after 
                                                                        ##each data generation 
    time.sleep(1) ##1 second sleep to prevent abnormal cloud interactions due to timing
  time.sleep(CHECK_DATA_EVERY_N_SECS)

Already generated datasets: ['0/', '1/', '2/']
Processing data for dataset number: 3
Writing into dir: pretraining_data_1024_embedded_mutformer/train/3
input_files: ./shards_train/shard_0 output_files: pretraining_data_1024_embedded_mutformer/train/3/shard_0.tfrecord
started
INFO:tensorflow:*** Reading from input files train ***
I0404 17:09:34.436829 140672754790784 create_pretraining_data.py:391] *** Reading from input files train ***
INFO:tensorflow:  ./shards_train/shard_0
I0404 17:09:34.437292 140672754790784 create_pretraining_data.py:393]   ./shards_train/shard_0
tokenizing inputs::  49% 73827/150533 [06:04<06:18, 202.40it/s]
Traceback (most recent call last):
  File "mutformer/create_pretraining_data.py", line 417, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.7/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/site-packages/absl/app.py", l